Import all the necessary libraries.

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification


from transformers import AdamW,XLNetTokenizer, DebertaTokenizer, DebertaForSequenceClassification,XLNetForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizerFast
from datasets import Dataset
import datasets
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [ ]:
# import re

In [ ]:
# from cleantext import clean

Load the data and check the split

In [ ]:
# test_data = pd.read_csv("/kaggle/input/amazon-ml-challennge/test.csv")

In [ ]:
# test_data.fillna(" ",inplace= True)
# test_data["desc"] = test_data["PRODUCT_TYPE_ID"].astype(str)+" "+test_data["TITLE"] + " "+ test_data["BULLET_POINTS"] + " " + test_data["DESCRIPTION"]
# test_data.drop(["PRODUCT_TYPE_ID","BULLET_POINTS","TITLE","DESCRIPTION"],axis=1,inplace=True)

In [ ]:
# data.desc.replace({r'[^\x00-\x7F|]+':''}, regex=True, inplace=True)

In [ ]:
#data.desc = data.desc.apply(lambda x: re.sub(r"\[|\]|\||;|:|\)|\(|,|-|\%", " ", x))
#test_data.desc = test_data.desc.apply(lambda x: re.sub(r"[^\w\s]","",x))
# data.desc.iloc[0].replace(r'[','')
# df['name'].str.replace(r"\(.*\)","")

In [ ]:
#test_data.head(2)

In [ ]:
#test_data.to_csv("Processed_test.csv",index=False)

In [ ]:
# data = pd.read_csv('/kaggle/input/amzn-cleaned/train.csv')


# data.desc.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [ ]:
# data[data.PRODUCT_TYPE_ID == 1650].desc.iloc[300]


In [ ]:
data = pd.read_csv("/kaggle/input/new-data/Processed_train.csv")
test_data = pd.read_csv("/kaggle/input/new-data/Processed_test.csv")
# test_data.desc.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [ ]:
data

In [ ]:
# data.apply(lambda row : str(row["PRODUCT_TYPE_ID"])+str(row['desc']), axis = 1)
# data.fillna("UNK",inplace=True)

In [ ]:
# data["new"] = data["PRODUCT_TYPE_ID"].astype(str) + " "+ data["desc"]

In [ ]:
# data

Split the data and create Dataset dict. Dataset dict is necessary for us to applyt .withtransform described later

In [ ]:
train, validation = train_test_split(data, test_size=0.2,random_state=42)
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validation)
ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds

In [ ]:
# test_data.fillna("UNK",inplace=True)

#test_data.apply(lambda row : str(row['desc']), axis = 1)
# test_data["new"] = test_data["PRODUCT_TYPE_ID"].astype(str) + " "+ test_data["desc"]

test_ds = Dataset.from_pandas(test_data)

In [ ]:
test_data

Check the dataset dict

In [ ]:
ds

Load Deberta default tokenizer

In [ ]:
#tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)
#tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

Check the tokenizer 

In [ ]:
# ds['train']

In [ ]:
# len(ds['train'][:100]['desc'])

In [ ]:
# train_encodings = tokenizer(ds['train'][:2]['desc'], max_length=128, truncation=True)

See the output

In [ ]:
# train_encodings

Decode the an encoder token collection

In [ ]:
# tokenizer.decode(train_encodings["input_ids"][1])

This function will take in a batch of data and return the tranform the data before returning the transformed data. This acts like a virtual dataset , which gets formed only when we access certain elements. The elements we access forms the batch. The batch will be defined by the user for example prepared_ds[:2] means accessing first two elements of the virtual dataset.

In [ ]:
def transform(batch):
    inputs = tokenizer(list(batch['desc']), padding='max_length', max_length=128, truncation=True)
    inputs['labels'] = np.log(batch['PRODUCT_LENGTH'])
    return inputs
prepared_ds = ds.with_transform(transform)

In [ ]:
def test_transform(batch):
    #print(batch["desc"])
    inputs = tokenizer(list(batch['desc']), padding='max_length', max_length=128, truncation=True)
    #inputs['labels'] = np.log(batch['PRODUCT_LENGTH'])
    return inputs

In [ ]:
# prepared_test_ds = test_ds.with_transform(test_transform)

Testing the virtual dataset.

In [ ]:
prepared_ds['train'][1]["input_ids"]

In [ ]:
tokenizer.decode(prepared_ds["train"][1]["input_ids"])

Gpu/cpu handler

In [ ]:
cuda=torch.device('cuda')
cpu=torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device==cuda:
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0)

The model.forward requires inputs of batch of data to be in the form of datapoints as rows stacked one top of another. This collate_fn does the online tranformation whenever dataloader asks for a batch of data.

In [ ]:
def collate_fn(batch):
    return {
        'input_ids': torch.stack([torch.tensor(x['input_ids']) for x in batch]).to(device),
        'token_type_ids': torch.stack([torch.tensor(x['token_type_ids']) for x in batch]).to(device),
        'attention_mask': torch.stack([torch.tensor(x['attention_mask']) for x in batch]).to(device),
        'labels': torch.tensor([x['labels'] for x in batch]).to(device)
    }

In [ ]:
def test_collate_fn(batch):
    return {
        'input_ids': torch.stack([torch.tensor(x['input_ids']) for x in batch]).to(device),
        'token_type_ids': torch.stack([torch.tensor(x['token_type_ids']) for x in batch]).to(device),
        'attention_mask': torch.stack([torch.tensor(x['attention_mask']) for x in batch]).to(device)
        #'labels': torch.tensor([x['labels'] for x in batch]).to(device)
    }

In [ ]:
# ds['validation']

In [ ]:
prepared_ds['validation']

Load only the needed data in the memory.

In [ ]:

train_dataloader = DataLoader(prepared_ds['train'],  batch_size=32,collate_fn=collate_fn ,drop_last=True)
validation_dataloader= DataLoader(prepared_ds['validation'],  batch_size=32,collate_fn=collate_fn ,drop_last=True)

In [ ]:
# test_loader  = DataLoader(prepared_test_ds,  batch_size=128,collate_fn=test_collate_fn ,drop_last=False)

Test the dataloader.

In [ ]:
# a=next(iter(train_dataloader))

Load the pretrained model. Here the model is Bert base.

In [ ]:

model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased",num_labels=1)
#model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base",num_labels=1)

#model = DebertaForSequenceClassification.from_pretrained("/kaggle/input/deberta-saved-model")

In [ ]:
# import torch.nn as nn
# model.classifier = nn.Linear(768,1,bias=True)
#model

In [ ]:
if device==cuda:
    model.cuda()

https://www.kaggle.com/code/prithaj08cs21s045/notebook17cc903d19Test the model.

In [ ]:
# model(a['input_ids'], token_type_ids=a['token_type_ids'], attention_mask=a['attention_mask'], labels=a['labels'][:,None])

Group the parameters , so that we don't apply weight decay to these parameters.

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if all(nd not in n for nd in no_decay)], 'weight_decay_rate': 0.01}, {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}]

Create the optimizer for bert.

In [ ]:
# %%capture 
# !pip install pytorch-pretrained-bert

In [ ]:
# from pytorch_pretrained_bert import BertAdam
#optimizer = BertAdam(optimizer_grouped_parameters,lr=2e-5,warmup=.1)
optimizer = AdamW(optimizer_grouped_parameters ,
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  # eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                 weight_decay=0.01,
                 correct_bias=False
                )

Helper function to check accuracy.

In [ ]:

# def accuracy(predicted_distribution, true_label):
#     predicted_label=np.argmax(predicted_distribution, axis=1)
#     return np.sum(predicted_label.flatten() == true_label.flatten()) / len(true_label.flatten())

Training loop.

In [ ]:
def train(epochs = 2):
    for _ in trange(epochs, desc="Epoch"):
      cumulative_loss = 0
      number_of_batch = 0
      for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        output=model.forward(batch['input_ids'], token_type_ids=batch['token_type_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'].float())
        loss = output.loss
        #print("1")
        loss.backward()
        optimizer.step()
        cumulative_loss += loss.item()
        number_of_batch += 1
        if (step%100==0):
          print(f"Train loss: {cumulative_loss / number_of_batch}")
        if number_of_batch>=5000:
            break
        
    model.save_pretrained('/kaggle/working/xlm-net_uncased_ml')
model.train()
train(epochs=1)

Load the model again for testing.

In [ ]:
# model = BertForSequenceClassification.from_pretrained('/kaggle/working/utsav/bert_uncased_quora')

In [ ]:
# model.cuda()


In [ ]:
# model(a['input_ids'], token_type_ids=a['token_type_ids'], attention_mask=a['attention_mask'], labels=a['labels'])

In [ ]:
def validate():
  model.eval()
  cumulated_accuracy = 0
  number_of_batch= 0
  for batch in validation_dataloader:
    with torch.no_grad():
      output=model.forward(batch['input_ids'], token_type_ids=batch['token_type_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
      logits = output.logits
    logits = logits.detach().cpu().numpy()
    label_ids = batch['labels'].to('cpu').numpy()
    batch_accuracy = np.mean(np.abs(logits -  label_ids))
    cumulated_accuracy += batch_accuracy
    if(number_of_batch%100 == 0):
        print (number_of_batch," batch acc: ", batch_accuracy)
    number_of_batch += 1
  print(f"Validation Accuracy: {cumulated_accuracy / number_of_batch}")

In [ ]:
# model.eval()
# validate()

In [ ]:
# from tqdm import tqdm as tqdm

In [ ]:
# prediction = []
# model.eval()
# for batch in tqdm(test_loader):
#     with torch.no_grad():
#         output=model.forward(batch['input_ids'], token_type_ids=batch['token_type_ids'], attention_mask=batch['attention_mask'])
#         logits = output.logits
#         prediction.append(np.exp(logits.cpu().numpy()))
        

In [ ]:
# prediction = np.vstack(prediction)

In [ ]:
# sample_submission = pd.read_csv("/kaggle/input/amazon-ml-challennge/sample_submission.csv")


In [ ]:
# sample_submission["PRODUCT_LENGTH"] = prediction 

In [ ]:

# sample_submission.to_csv("prediction_deberta_10k.csv",index=False)

In [ ]:
# sample_submission.head(5)